In [1]:
import detectron2
from detectron2.utils.logger import setup_logger

# import some common libraries
import numpy as np
import cv2
import random
#from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import torch

from tqdm import tqdm
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from matplotlib.pyplot import imshow
from PIL import Image
import os
import numpy as np
import json
from detectron2.structures import BoxMode

from detectron2.config import get_cfg

from detectron2.data import DatasetCatalog, MetadataCatalog

from detectron2.modeling import build_model

from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import csv 
from defaults import DefaultBatchPredictor
######################################################################################
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--data_dir', type=str, default=None)
parser.add_argument('--model_dir', type=str, default=None)

parser.add_argument('--batch', type=int, default=24)

parser.add_argument('--num_of_data', type=int, default=42000)
parser.add_argument('--data_random', type=bool, default=False)

parser.add_argument('-f')
args = parser.parse_args()

######################################################################################

data_path = '/root/notebooks/final/caltech_origin_data_refine/'
model_path = '/root/notebooks/final/detectron2_out_part3_25p/'

num_of_data = 4000

data_random_select = args.data_random

batch = 5

def get_pedestrain_dict(image_list, json_list):
    dataset_dicts = []
    
    for i,path in tqdm(enumerate(image_list)):
        filename = path
        #img = cv2.imread(path)
        # height, width = cv2.imread(filename).shape[:2]
        record = {}
        record['file_name'] = filename
        #record['file_img'] = img
        record['image_id'] = i #path.split('/')[-1][:-5]
        #id is like 000000 or 000001
        record['height']= 480
        record['width']= 640
        
        #for i in data_list[1] to get bbox and category
        objs = []
        
        people = json_list[i]
        with open(people) as p:
            json_context = json.load(p)
            for person in json_context:
                boxes = list(map(float, person['pos']))
                obj = {
                    "bbox": boxes,
                    "bbox_mode": BoxMode.XYWH_ABS,
                    #"segmentation": [poly], To draw a line, along to ballon
                    "category_id": 0,
                    "iscrowd": 0
                }
                objs.append(obj)
            record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts #list of dicts


def take_path(path):
    tmp = []
    for json_number in tqdm(sorted(os.listdir(path))):
        if json_number == '.ipynb_checkpoints':
            pass
        else: tmp.append(os.path.join(path, json_number))
    return tmp


data_pathlist_json = take_path(data_path+'street_json/')

data_pathlist_json = data_pathlist_json[42000:(42000+num_of_data)]

train_people_jsonlist = data_pathlist_json
random.shuffle(train_people_jsonlist)
train_people_imagelist = [i.replace('street_json', 'street').replace('/json/', '/output/').replace('.json', '.jpg') for i in train_people_jsonlist]

dataset_dicts = get_pedestrain_dict(train_people_imagelist, train_people_jsonlist)

#print(dataset_dicts)
print("Total images : "+str(len(train_people_imagelist)))


cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("Base-RCNN-FPN.yaml"))
cfg.DATALOADER.NUM_WORKERS = 4
cfg.SOLVER.IMS_PER_BATCH = 3
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER =  50000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1 
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7

cfg.MODEL.WEIGHTS = os.path.join('/root/notebooks/final/', "model_final.pth")

#pre = DefaultPredictor(cfg)
                            
#loader = pre.create_batch_loader(train_people_imagelist, train_people_jsonlist, batch)



100%|██████████| 110913/110913 [00:00<00:00, 682894.62it/s]
4000it [00:05, 755.22it/s]


Total images : 4000


In [2]:
# result, result_dic = pre.batchPredictor()
    
# print(result[1])
# print(result_dic[1])

In [3]:
pre2 = DefaultPredictor(cfg)

In [4]:
iou = 0.5
def compute_TF(outputs, d, iou=0.5):
    TF_list = []
    for box in outputs['instances'].pred_boxes.to("cpu"):
        
        for annotation in d['annotations']:
            curr_iou = 0
            flag = False
            
            top, left = annotation['bbox'][0], annotation['bbox'][1]
            bottom, right = top+annotation['bbox'][2], left+annotation['bbox'][3] 
            
            curr_iou = compute_iou((top, left, bottom, right), box)
            
            if curr_iou >= iou:
                TF_list.append('T')
                flag = True
                break
        if flag == False:
            TF_list.append('F')
    return TF_list

def compute_iou(rec1, rec2):
    """
    computing IoU
    :param rec1: (y0, x0, y1, x1), which reflects
            (top, left, bottom, right)
    :param rec2: (y0, x0, y1, x1)
    :return: scala value of IoU
    """
    # computing area of each rectangles
    S_rec1 = (rec1[2] - rec1[0]) * (rec1[3] - rec1[1])
    S_rec2 = (rec2[2] - rec2[0]) * (rec2[3] - rec2[1])
 
    # computing the sum_area
    sum_area = S_rec1 + S_rec2
 
    # find the each edge of intersect rectangle
    left_line = max(rec1[1], rec2[1])
    right_line = min(rec1[3], rec2[3])
    top_line = max(rec1[0], rec2[0])
    bottom_line = min(rec1[2], rec2[2])
 
    # judge if there is an intersect
    if left_line >= right_line or top_line >= bottom_line:
        return 0
    else:
        intersect = (right_line - left_line) * (bottom_line - top_line)
        return (intersect / (sum_area - intersect))*1.0

In [5]:
d = dataset_dicts

x = []
y = []

MR_list = []
FP_list = []

In [6]:
len(d[0]['annotations'])

2

In [7]:
for i in range(200):
    
    outputs = pre2(cv2.imread(train_people_imagelist[i]))
    
    d = dataset_dicts[i]
    
    TP = 0
    
    TF_list = compute_TF(outputs, d, iou)
    for TF in TF_list:
        if TF == 'T':
            TP += 1
    if len(d['annotations']) > TP:
        MR = (len(d['annotations']) - TP) / len(d['annotations'])
        FP = len(outputs['instances']) - TP
    else: 
        MR = 0
        FP = len(outputs['instances']) - len(d['annotations'])

    MR_list.append(MR)
    FP_list.append(FP)
    #break
        
x.append(sum(FP_list)/len(FP_list))
y.append(sum(MR_list)/len(MR_list))

In [8]:
print(x,y)

[0.31] [0.8885694444444446]


In [70]:
cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("Base-RCNN-FPN.yaml"))
cfg.DATALOADER.NUM_WORKERS = 4
cfg.SOLVER.IMS_PER_BATCH = 3
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER =  50000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1 
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7

cfg.MODEL.WEIGHTS = os.path.join('/root/notebooks/final/', "model_final.pth")

In [65]:
resu = []
x = []
y = []

count = 1

gap = 5
for threshold in range(30,105,gap):
    print("Threshold test : ",threshold/100.0," (" ,count,'/',len(range(40,105,gap)), ")")
    count+=1
#     x,y = compuete_MR(threshold/100.0)
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = threshold/100.0
    
    pre2 = DefaultBatchPredictor(cfg)
    
    MR_list = []
    FP_list = []
    
    for i in range(100):
    
        outputs = pre2(cv2.imread(train_people_imagelist[i]))

        d = dataset_dicts[i]

        TP = 0

        TF_list = compute_TF(outputs, d, iou)
        for TF in TF_list:
            if TF == 'T':
                TP += 1
        if len(d['annotations']) > TP:
            MR = (len(d['annotations']) - TP) / len(d['annotations'])
            FP = len(outputs['instances']) - TP
        else: 
            MR = 0
            FP = len(outputs['instances']) - len(d['annotations'])

        MR_list.append(MR)
        FP_list.append(FP)
    x = sum(FP_list)/len(FP_list)
    y = sum(MR_list)/len(MR_list)
    resu.append([x,y])

Threshold test :  0.3  ( 1 / 13 )


KeyboardInterrupt: 

In [11]:
print(resu)

NameError: name 'resu' is not defined

In [ ]:
with open(os.path.join('/root/notebooks/final/', "model_result_D.csv") , 'w+', newline ='') as f:
    write = csv.writer(f) 
    write.writerows(resu)

In [ ]:
!python plot.py --csv_path '/root/notebooks/final/model_result_D.csv' --output_dir '/root/notebooks/final/'



In [ ]:
Image.open('/root/notebooks/final/plot.jpg')

In [23]:
pre2 = DefaultBatchPredictor(cfg)

for i in range(3):
    outputs = pre2(cv2.imread(train_people_imagelist[i]))
    print(outputs)

Load weight from :  /root/notebooks/final/model_final.pth
{'instances': Instances(num_instances=2, image_height=480, image_width=640, fields=[pred_boxes: Boxes(tensor([[342.4135, 190.9354, 376.8608, 262.4553],
        [231.9504, 174.6648, 278.0927, 275.0215]], device='cuda:0')), scores: tensor([0.4861, 0.3432], device='cuda:0'), pred_classes: tensor([0, 0], device='cuda:0')])}
{'instances': Instances(num_instances=2, image_height=480, image_width=640, fields=[pred_boxes: Boxes(tensor([[152.7999, 151.3908, 179.3864, 208.8824],
        [459.0749, 207.8614, 513.2574, 320.2764]], device='cuda:0')), scores: tensor([0.7124, 0.3711], device='cuda:0'), pred_classes: tensor([0, 0], device='cuda:0')])}
{'instances': Instances(num_instances=4, image_height=480, image_width=640, fields=[pred_boxes: Boxes(tensor([[599.9340, 197.8654, 640.0000, 231.0118],
        [556.5503, 159.6331, 586.8049, 223.9113],
        [140.0022, 184.6031, 169.6389, 248.7350],
        [  0.7024, 186.7161,  39.8138, 227.646

In [105]:
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.95

pre2 = DefaultBatchPredictor(cfg)

Load weight from :  /root/notebooks/final/model_final.pth


In [106]:
_ = pre2.create_batch_loader(train_people_imagelist[:20], train_people_jsonlist[:20])

20it [00:00, 587.49it/s]


In [107]:
r1, r2 = pre2.batchPredictor()

100%|██████████| 1/1 [00:01<00:00,  1.85s/it]


In [108]:
r1

[{'instances': Instances(num_instances=0, image_height=480, image_width=640, fields=[pred_boxes: Boxes(tensor([], device='cuda:0', size=(0, 4))), scores: tensor([], device='cuda:0'), pred_classes: tensor([], device='cuda:0', dtype=torch.int64)])},
 {'instances': Instances(num_instances=0, image_height=480, image_width=640, fields=[pred_boxes: Boxes(tensor([], device='cuda:0', size=(0, 4))), scores: tensor([], device='cuda:0'), pred_classes: tensor([], device='cuda:0', dtype=torch.int64)])},
 {'instances': Instances(num_instances=0, image_height=480, image_width=640, fields=[pred_boxes: Boxes(tensor([], device='cuda:0', size=(0, 4))), scores: tensor([], device='cuda:0'), pred_classes: tensor([], device='cuda:0', dtype=torch.int64)])},
 {'instances': Instances(num_instances=0, image_height=480, image_width=640, fields=[pred_boxes: Boxes(tensor([], device='cuda:0', size=(0, 4))), scores: tensor([], device='cuda:0'), pred_classes: tensor([], device='cuda:0', dtype=torch.int64)])},
 {'insta

In [109]:
MR_list = []
FP_list = []

for i in range(len(r1)):
    outputs = r1[i]
    d = dataset_dicts[i]
    
    TP = 0

    TF_list = compute_TF(outputs, d, iou)
    for TF in TF_list:
        if TF == 'T':
            TP += 1
    if len(d['annotations']) > TP:
        MR = (len(d['annotations']) - TP) / len(d['annotations'])
        FP = len(outputs['instances']) - TP
    else: 
        MR = 0
        FP = len(outputs['instances']) - len(d['annotations'])

    MR_list.append(MR)
    FP_list.append(FP)
x = sum(FP_list)/len(FP_list)
y = sum(MR_list)/len(MR_list)
resu.append([x,y])

In [110]:
resu

[[3.25, 0.6658333333333334],
 [0.75, 0.8408333333333335],
 [0.3, 0.8758333333333332],
 [0.2, 0.9316666666666666],
 [0.0, 1.0],
 [0.05, 0.9541666666666666]]

In [111]:
def compute_XY(r, r_dic):
    MR_list =[]
    FP_list = []
    
    TP = 0
    for i in range(len(r)):
        outputs = r[i]
        d = dataset_dicts[i]

        TP = 0

        TF_list = compute_TF(outputs, d, iou)
        for TF in TF_list:
            if TF == 'T':
                TP += 1
        if len(d['annotations']) > TP:
            MR = (len(d['annotations']) - TP) / len(d['annotations'])
            FP = len(outputs['instances']) - TP
        else: 
            MR = 0
            FP = len(outputs['instances']) - len(d['annotations'])

        MR_list.append(MR)
        FP_list.append(FP)
    x = sum(FP_list)/len(FP_list)
    y = sum(MR_list)/len(MR_list)
    print('X:',x)
    print('Y:',y)
    return x,y

In [112]:
com(r1,dataset_dicts)

(0.05, 0.9541666666666666)